In [1]:
import pandas as pd 
import numpy as np
from pandas import Series,DataFrame
import torch
import datetime
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from pyod.models.abod import ABOD
from pyod.models.copod import COPOD
from pyod.models.hbos import HBOS
from pyod.models.sod  import SOD
from pyod.models.rod import ROD
import sys
from embedding import Graph2Vec
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import os
import datetime
import glob
import os 
import pandas as pd
from sklearn import preprocessing
from __future__ import (absolute_import, division, print_function)

from mpl_toolkits.basemap import Basemap, shiftgrid, maskoceans, interp
os.chdir("/Users/mikegu/Desktop/weather data exploration/jiuji")

In [2]:
def data(file,area):
    Time = pd.read_csv(file)
    Area = Time[Time['지점'] == area]
    Area = Area.dropna(axis = 0)
    return Area

#i=1
def data1(area):
    
    DF10 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2010.csv',area)
    DF11 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2011.csv',area)
    DF12 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2012.csv',area)
    DF13 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2013.csv',area)
    DF14 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2014.csv',area)
    DF15 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2015.csv',area)
    DF16 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2016.csv',area)
    DF17 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2017.csv',area)
    DF18 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2018.csv',area)
    DF19 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2019.csv',area)
    DF20 = data('/Users/mikegu/Desktop/weather data exploration/koreaweather/OBS_ASOS_TIM_2020.csv',area)
    df = pd.concat([DF10,DF11,DF12,DF13,DF14,DF15,DF16,DF17,DF18,DF19,DF20])
    return df

In [44]:
chengshizidian = {
106: 'Donghae',
 108: 'Seoul',
 112: 'Incheon',
 114: 'Wonju',
 115: 'Ulleung do',
 119: 'Suwon',
 121: 'Yeongwol',
 127: 'Chungju',
 129: 'Seosan',
 130: 'Uljin',
 131: 'Cheongju',
 133: 'Daejeon',
 135: 'Chupungnyeong',
 136: 'Andong',
 137: 'Sangju',
 138: 'Pohang',
 140: 'Gunsan',
 143: 'Daegu',
 146: 'Jeonju',
 152: 'Ulsan',
 155: 'Changwon',
 156: 'Gwangju',
 159: 'Busan',
 162: 'Tongyeong',
 165: 'Mokpo',
 168: 'Yeosu',
 170: 'Wando',
 184: 'Jeju',
 189: 'Seogwipo',
 192: 'Jinju',
 201: 'Ganghwa',
 202: 'Yangpyeong',
 203: 'Icheon',
 211: 'Inje',
 212: 'Hongcheon',
 216: 'Taebaek',
 221: 'Jecheon',
 226: 'Boeun',
 235: 'Boryeong',
 236: 'Buyeo',
 238: 'Geumsan',
 243: 'Buan',
 244: 'Imsil',
 245: 'Jeongeup',
 247: 'Namwon',
 248: 'Jangsu',
 254: 'Sunchang',
 257: 'Yangsan',
 259: 'Gangjin',
 260: 'Jangheung',
 261: 'Haenam',
 262: 'Goheung',
 271: 'Bonghwa',
 272: 'Yeongju',
 273: 'Mungyeong',
 277: 'Yeongdeok',
 278: 'Uiseong',
 279: 'Gumi',
 281: 'Yeongcheon',
 284: 'Geochang',
 288: 'Miryang',
 289: 'Sancheong',
 294: 'Geoje',
 295: 'Namhae'}

In [45]:
len(chengshizidian)

64

In [35]:
for key,value in chengshizidian.items():    
    df=data1(int(key))
    #df.to_csv("/Users/mikegu/Desktop/weather data exploration/zhongjitest/test.csv",encoding="utf-8-sig")
    df1=df.fillna(method="ffill")
    riqi=np.array(df1['일시'].values)
    riqilist=riqi.tolist()
    start='2010/1/1'
    end='2020/12/31'
    datestart=datetime.datetime.strptime(start,'%Y/%m/%d')
    dateend=datetime.datetime.strptime(end,'%Y/%m/%d')

    
    data_list = list()
    while datestart<=dateend:
        data_list.append(datestart.strftime("%Y/%-m/%-d")) 
        datestart+=datetime.timedelta(days=1)
    data_list[0]
    filllist = []
    for i in data_list:
        for j in range(0,24):
            times = i+" "+str(j)+":00"
            if times in riqilist:
                #print("ture")
                filllist.append("true")
            else:
                #print(times)
                filllist.append(times)
    for i, line in enumerate(filllist):
        if line != 'true':
            d= [[] * 12]
            pd_arr1 = df1[:i]
            pd_arr2 = df1[i:]
            df1 = pd_arr1.append(d, ignore_index = True).append(pd_arr2, ignore_index=True)
    df2 = df1.interpolate(method='linear', limit_direction='forward')
    df2 = df2.fillna(method="bfill")
    seoul1017 = df2.iloc[:,3:]
    tianqidf = df2["기온(°C)"]
    shidudf = df2['습도(%)']
    zhengqiyadf = df2['증기압(hPa)']
    ludianwendudf = df2['이슬점온도(°C)']
    dangdiqiyadf = df2['현지기압(hPa)']
    haimianqiyadf = df2['해면기압(hPa)']
    dangdiwendudf = df2['지면온도(°C)']
    tianqi=tianqidf.values 
    shidu=shidudf.values
    zhengqiya=zhengqiyadf.values
    ludianwendu=ludianwendudf.values
    dangdiqiya=dangdiqiyadf.values
    haimianqiya=haimianqiyadf.values
    dangdiwendu=dangdiwendudf.values
    Tianqi=[]
    for i in range(int(len(tianqi)/24)):
        sum1=0
        for j in range(24):
            sum1=sum1+tianqi[j+(i*24)]
        Tianqi.append(sum1/24)
    tianqi1=np.array(Tianqi)  
    tianqi2=np.around(tianqi1,5)  
    Shidu=[]
    for i in range(int(len(shidu)/24)):
        sum2=0
        for j in range(24):
            sum2=sum2+shidu[j+(i*24)]
        Shidu.append(sum2/24)
    shidu1=np.array(Shidu)  
    shidu2=np.around(shidu1,5) 
    Zhengqiya=[]
    for i in range(int(len(zhengqiya)/24)):
        sum3=0
        for j in range(24):
            sum3=sum3+zhengqiya[j+(i*24)]
        Zhengqiya.append(sum3/24)
    zhengqiya1=np.array(Zhengqiya)
    zhengqiya2=np.around(zhengqiya1,5)
    Ludianwendu=[]
    for i in range(int(len(ludianwendu)/24)):
        sum4=0
        for j in range(24):
            sum4=sum4+ludianwendu[j+(i*24)]
        Ludianwendu.append(sum4/24)
    ludianwendu1=np.array(Ludianwendu)
    ludianwendu2=np.around(ludianwendu1,5)
    Dangdiqiya=[]
    for i in range(int(len(dangdiqiya)/24)):
        sum5=0
        for j in range(24):
            sum5=sum5+dangdiqiya[j+(i*24)]
        Dangdiqiya.append(sum5/24)
    dangdiqiya1=np.array(Dangdiqiya)
    dangdiqiya2=np.around(dangdiqiya1,5)
    Haimianqiya=[]
    for i in range(int(len(haimianqiya)/24)):
        sum6=0
        for j in range(24):
            sum6=sum6+haimianqiya[j+(i*24)]
        Haimianqiya.append(sum6/24)
    haimianqiya1=np.array(Haimianqiya)
    haimianqiya2=np.around(haimianqiya1,5)
    Dangdiwendu=[]
    for i in range(int(len(dangdiwendu)/24)):
        sum7=0
        for j in range(24):
            sum7=sum7+dangdiwendu[j+(i*24)]
        Dangdiwendu.append(sum7/24)
    dangdiwendu1=np.array(Dangdiwendu)
    dangdiwendu2=np.around(dangdiwendu1,5)
    city=value
    City=[]
    for i in range(len(Tianqi)):
        City.append(city)
    def processingforcorr(data):
            Corr = data.corr().values
            Corr = np.abs(Corr)
            Corr = pd.DataFrame(Corr)
            return Corr.fillna(0).values

    def Graph(data):
        maxlag = 1
        test = 'ssr_chi2test'
        X = []
        new = processingforcorr(data.iloc[:24])
        X.append(new)
        i = 1
        while i<=data.shape[0]-25:
            new = processingforcorr(data.iloc[i:i+24])
            X.append(new)
            i=i+24
                
        return np.array(X)

    def entropy(X):
            E = []
            for i in range(X.shape[0]):
                P = []
                for j in range(X.shape[1]):
                    if i !=j:
                        e = -X[i][j]*np.log(X[i][j])
                        P.append(e)
                P = np.array(P)
                E.append(np.sum(P))
            return np.array(E)

    def graphentropy(X):
            E = []
            for i in range(X.shape[0]):
                e = entropy(X[i])
                E.append(np.sum(e))
            return np.array(E)

    def distance(x,y):
            distance = np.mean(np.power((x - y),2))
            return distance

    def getMatrix(data,E):
            Matrix = []
            for i in range(E.shape[0]):
                dis = []
                for j in range(len(E)):
                    dis.append(distance(E[i],E[j]))
                dis = np.array(dis)
                index = np.argsort(dis)[1]
                Matrix.append(data[index])
                
            return np.array(Matrix)
    x = Graph(seoul1017)
    X = x.reshape(-1,49)
    pca = PCA(n_components=4*4)
    X= pca.fit_transform(X)
    for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                if X[i][j] == 0:
                    X[i][j] = 1
    X = np.abs(X.reshape(X.shape[0],4,4))


    E = graphentropy(X)

    MatrixX = getMatrix(X,E)

    TrainXTensor = torch.from_numpy(X.reshape(X.shape[0], 16)).type(torch.FloatTensor)
    TrainSTensor = torch.from_numpy(MatrixX.reshape(X.shape[0], 16)).type(torch.FloatTensor)
            
    model = torch.load('/Users/mikegu/Desktop/weather data exploration/Model.pth')
    model.eval()
    ed1, ed2, de1, de2 = model(TrainXTensor, TrainSTensor)
    Embed = ed1.data.numpy()

    df_Embed = pd.DataFrame(Embed)
    df_Embed.columns = ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']

    start='2010-01-01'
    end='2020-12-31'
    
    datestart=datetime.datetime.strptime(start,'%Y-%m-%d')
    dateend=datetime.datetime.strptime(end,'%Y-%m-%d')
    
    data_list = list()
    while datestart<=dateend:
        data_list.append(datestart.strftime('%Y-%m-%d')) 
        datestart+=datetime.timedelta(days=1)
    #lof
    lof = LocalOutlierFactor(n_neighbors=10, contamination=0.2, algorithm='auto', n_jobs=-1, novelty=True)
    lof.fit(Embed)
    lofscore=np.abs(lof.negative_outlier_factor_)
    normalized_lofscore = preprocessing.normalize([lofscore])
    normalized_lofscore = np.around(normalized_lofscore[0],5)
    #if
    isolation_forest = IsolationForest(n_estimators=100,contamination=0.05) 
    isolation_forest.fit(Embed)
    Ifscore = np.around(isolation_forest.decision_function(Embed),5) 
    normalized_Ifscore = preprocessing.normalize([Ifscore])
    normalized_Ifscore = np.around(normalized_Ifscore[0],5)
    #copod
    copod=COPOD()
    copod.fit(Embed)
    copodscore=np.around(copod.decision_scores_,5)
    normalized_copodscore = preprocessing.normalize([copodscore])
    normalized_copodscore = np.around(normalized_copodscore[0],5)
    #abod
    abod=ABOD(contamination=0.1,method='fast',n_neighbors=20)
    abod.fit(Embed)
    abodscore=np.abs(abod.decision_scores_)
    normalized_abodscore = preprocessing.normalize([abodscore])
    normalized_abodscore = np.around(normalized_abodscore[0],5)
    #hbos
    hbos=HBOS()
    hbos.fit(Embed)
    hbosscore=hbos.decision_scores_
    normalized_hbosscore = preprocessing.normalize([hbosscore])
    normalized_hbosscore = np.abs(normalized_hbosscore)
    normalized_hbosscore = np.around(normalized_hbosscore[0],5)
    #sod
    sod=SOD()
    sod.fit(Embed)
    sodscore=sod.decision_scores_
    normalized_sodscore=preprocessing.normalize([sodscore])
    normalized_sodscore=np.around(normalized_sodscore[0],5)
    #rod
    rod=ROD()
    rod.fit(Embed)
    rodscore=rod.decision_scores_
    normalized_rodscore=preprocessing.normalize([rodscore])
    normalized_rodscore=np.around(normalized_rodscore[0],5)
    dic={'City':City,'Date':data_list,'Temperature(°C)':tianqi2,'Humidity(%)':shidu2,'Vapor pressure(hPa)':zhengqiya2,'Dew point temperature(°C)':ludianwendu2,'Local air pressure(hPa)':dangdiqiya2,'Sea surface pressure(hPa)':haimianqiya2,'Ground temperature(°C)':dangdiwendu2,'Anoamly Score(LOF)':normalized_lofscore,'Anomaly Score(IF)':normalized_Ifscore,'Anomaly Score(COPOD)':normalized_copodscore,'Anomaly Score(ABOD)':normalized_abodscore,'Anomaly Score(HBOS)':normalized_hbosscore,'Anomaly Score(SOD)':normalized_sodscore,'Anomaly Score(ROD)':normalized_rodscore}
    finaldf = pd.DataFrame.from_dict(dic)
    finaldf.to_csv(value+"2010to2020score.csv",mode='a',index=False,header=True,encoding='utf-8-sig')

/var/folders/3r/stx0z6fn3y7gn9nz8mwg1xfr0000gn/T/ipykernel_60142/3748958021.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = pd_arr1.append(d, ignore_index = True).append(pd_arr2, ignore_index=True)
/var/folders/3r/stx0z6fn3y7gn9nz8mwg1xfr0000gn/T/ipykernel_60142/3748958021.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = pd_arr1.append(d, ignore_index = True).append(pd_arr2, ignore_index=True)
/var/folders/3r/stx0z6fn3y7gn9nz8mwg1xfr0000gn/T/ipykernel_60142/3748958021.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = pd_arr1.append(d, ignore_index = True).append(pd_arr2, ignore_index=True)
/var/folders/3r/stx0z6fn3y7gn9nz8mwg1xfr0000gn/T/ipykernel_60142/3748958021.py:33: FutureWarning: The fram

In [33]:
import pandas
pandas.__version__

'1.5.1'